In [1]:
import numpy as np # linear algebra
import pandas as pd

In [2]:
from sklearn import svm

from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold

from sklearn import preprocessing

from sklearn.metrics import f1_score

In [3]:
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")
sample.head(2)

,id,target
0,0,0
1,2,0


In [4]:
train_limpio = pd.read_csv("train_limpio_con_BOW_de_2500_y_Stemming.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,aba,abandon,abc,abcnew,abl,ablaz,about,abov,absolut,abstorm,...,wildfir-keyword,windstorm-keyword,wound-keyword,wreck-keyword,wreckag-keyword,isRealPlace,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,70,2,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,40,1,0.1,0.4


In [5]:
test_limpio = pd.read_csv("test_limpio_con_BOW_de_2500_y_Stemming.csv",encoding = "ISO-8859-1")
test_limpio.head()

,aba,abandon,abc,abcnew,abl,ablaz,about,abov,absolut,abstorm,...,wildfir-keyword,windstorm-keyword,wound-keyword,wreck-keyword,wreckag-keyword,isRealPlace,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,40,2,-1.00,1.00
1,0,0,0,0,0,0,1,0,0,0,...,False,False,False,False,False,False,70,2,0.25,0.55
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,100,2,0.00,0.00
3,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,40,1,0.00,0.00
4,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,50,2,0.00,0.00


In [6]:
train_y=train_limpio.target

In [7]:
train_limpio = train_limpio.drop(columns = ['target'])

In [8]:
train_limpio_normalizado = train_limpio.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(train_limpio_normalizado)
train_limpio_normalizado = pd.DataFrame(x_scaled)

In [9]:
test_limpio_normalizado = test_limpio.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(test_limpio_normalizado)
test_limpio_normalizado = pd.DataFrame(x_scaled)

In [130]:
x_train, x_test, y_train, y_test = train_test_split(train_limpio_normalizado, train_y, test_size=0.2)

In [131]:
clf = svm.SVC()
clf.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [132]:
y_pred = clf.predict(x_test)

f1score = f1_score(y_test, y_pred)
print(f"Model Score: {f1score * 100} %")

Model Score: 76.76056338028168 %


In [13]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [134]:
training_error=[]
test_error=[]

for train_index,test_index in kf.split(train_limpio_normalizado):
    print('iter')
    x_train,x_test = train_limpio_normalizado.loc[train_index], train_limpio_normalizado.loc[test_index]
    y_train,y_test = train_y[train_index], train_y[test_index]
    
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    
    #error de entrenamiento:
    pred = clf.predict(x_train)
    training_error.append(f1_score(y_train.values, pred, average='macro'))
    
    
        
    #error de test:
    pred = clf.predict(x_test)
    test_error.append(f1_score(y_test.values, pred, average='macro'))
    
print('Error de entrenamiento:',training_error)
print('Error de test:',test_error)

iter
iter
iter
iter
iter
Error de entrenamiento: [0.9288016949428641, 0.9352246030036342, 0.9307978723945535, 0.9317218887646183, 0.9294494209070807]
Error de test: [0.7642922911156823, 0.7451959847036329, 0.7239359572326771, 0.7232840880433062, 0.7827675153973228]


In [10]:
# Tunning

In [15]:
c=[1, 10, 100, 1000]

for i in c:
    training_error=[]
    test_error=[]

    for train_index,test_index in kf.split(train_limpio_normalizado):
        print('iter')
        x_train,x_test = train_limpio_normalizado.loc[train_index], train_limpio_normalizado.loc[test_index]
        y_train,y_test = train_y[train_index], train_y[test_index]

        clf = svm.SVC(C=i,kernel='linear')
        clf.fit(x_train, y_train)

        #error de entrenamiento:
        pred = clf.predict(x_train)
        training_error.append(f1_score(y_train.values, pred, average='macro'))



        #error de test:
        pred = clf.predict(x_test)
        test_error.append(f1_score(y_test.values, pred, average='macro'))

    print('Error de entrenamiento:',training_error)
    print('Error de test:',test_error)


iter
iter
iter
iter
iter
Error de entrenamiento: [0.9154378188866442, 0.9271545300375148, 0.9237955311543288, 0.9229971808194587, 0.9206298526868125]
Error de test: [0.7379743472928948, 0.7165322433487828, 0.7243931739840448, 0.7010311041350932, 0.7449074570433953]
iter
iter
iter
iter
iter
Error de entrenamiento: [0.9622416717364453, 0.9644151144422797, 0.9637843807512156, 0.9636212133326526, 0.9593259409368056]
Error de test: [0.6879098360655738, 0.6914848609791513, 0.6821078704219081, 0.6374081617807521, 0.6859677752143004]
iter
iter
iter
iter
iter
Error de entrenamiento: [0.9778070583237588, 0.9790539287661881, 0.9815437197474691, 0.9802506525980271, 0.9778723340767541]
Error de test: [0.6498467637349024, 0.664296350481683, 0.655225067738834, 0.6492326787263012, 0.6760274385340335]
iter
iter
iter
iter
iter
Error de entrenamiento: [0.9804850561673952, 0.9809017049325791, 0.9832408966485824, 0.9827603656113901, 0.9804071441564953]
Error de test: [0.6354898848663493, 0.6782830281509369

In [17]:
x=[[1,0.001],[1,0.0001],[10,0.001],[10,0.0001],[100,0.001],[100,0.0001],[1000,0.001],[1000,0.0001]]

for i in x:
    training_error=[]
    test_error=[]
    print(i)
    for train_index,test_index in kf.split(train_limpio_normalizado):
        print('iter')
        x_train,x_test = train_limpio_normalizado.loc[train_index], train_limpio_normalizado.loc[test_index]
        y_train,y_test = train_y[train_index], train_y[test_index]

        clf = svm.SVC(C=i[0],gamma=i[1])
        clf.fit(x_train, y_train)

        #error de entrenamiento:
        pred = clf.predict(x_train)
        training_error.append(f1_score(y_train.values, pred, average='macro'))



        #error de test:
        pred = clf.predict(x_test)
        test_error.append(f1_score(y_test.values, pred, average='macro'))

    print('Error de entrenamiento:',training_error)
    print('Error de test:',test_error)

[1, 0.001]
iter
iter
iter
iter
iter
Error de entrenamiento: [0.5895916096469676, 0.6237372216929287, 0.5575983129450849, 0.5971716326222111, 0.5401656243045714]
Error de test: [0.5832002409735234, 0.5259419279907085, 0.39271438730017794, 0.47352507847030934, 0.43635516007858566]
[1, 0.0001]
iter
iter
iter
iter
iter
Error de entrenamiento: [0.35800126502213786, 0.36150136296917595, 0.3682572614107884, 0.36126258389261745, 0.36683991683991685]
Error de test: [0.383151073309032, 0.36988001654944147, 0.34211663066954645, 0.37081438610996276, 0.3481798715203426]
[10, 0.001]
iter
iter
iter
iter
iter
Error de entrenamiento: [0.8202795632375794, 0.8268043886328278, 0.819747677221876, 0.8210055210254532, 0.8204181126164811]
Error de test: [0.7423605133030371, 0.7288794331154784, 0.6957981259549713, 0.6928563327004291, 0.7635283936833472]
[10, 0.0001]
iter
iter
iter
iter
iter
Error de entrenamiento: [0.5959586885321828, 0.6280563402385436, 0.5635153701966001, 0.6021612240968435, 0.54889894701847

KeyboardInterrupt: 

In [ ]:
# El maximo fue con: [100, 0.001], veo por sus bordes

In [19]:
x=[[80, 0.001],[100, 0.001], [120, 0.001]]

for i in x:
    training_error=[]
    test_error=[]
    print(i)
    for train_index,test_index in kf.split(train_limpio_normalizado):
        x_train,x_test = train_limpio_normalizado.loc[train_index], train_limpio_normalizado.loc[test_index]
        y_train,y_test = train_y[train_index], train_y[test_index]

        clf = svm.SVC(C=i[0],gamma=i[1])
        clf.fit(x_train, y_train)

        #error de entrenamiento:
        pred = clf.predict(x_train)
        training_error.append(f1_score(y_train.values, pred, average='macro'))



        #error de test:
        pred = clf.predict(x_test)
        test_error.append(f1_score(y_test.values, pred, average='macro'))

    print('Error de entrenamiento:',training_error)
    print('Error de test:',test_error)

[80, 0.001]
Error de entrenamiento: [0.871920203407629, 0.8851840696117804, 0.8777675885406698, 0.8808512203213712, 0.8766290828343981]
Error de test: [0.7703612366684872, 0.7278236634378985, 0.744703257237507, 0.7188505557064736, 0.785893105749009]
[100, 0.001]
Error de entrenamiento: [0.8789371473016292, 0.8892331855400755, 0.8831356938515942, 0.8865478527726429, 0.8823350188433696]
Error de test: [0.7641355135774209, 0.7223966431818352, 0.7462678015464737, 0.7217267648552566, 0.7839355682280164]
[120, 0.001]
Error de entrenamiento: [0.8845416240930273, 0.8959675265418772, 0.8877426018559915, 0.8915148940066642, 0.8878394803704857]
Error de test: [0.7618337839227625, 0.7135965416882095, 0.7444051192712682, 0.7248660228170438, 0.7820445915719152]


BOW de 500:
Error de entrenamiento: [0.9036035333541272, 0.9093469488061671, 0.9055951846656467, 0.908319383971707, 0.9056325432275374]
Error de test: [0.7523721038267233, 0.7313520975534933, 0.715298389537034, 0.6860331850461006, 0.7619960184657876]
    
BOW de 1000:
Error de entrenamiento: [0.9144565000501346, 0.9202991537002544, 0.9164692765091506, 0.918243887496519, 0.9168028275156694]
Error de test: [0.7615579274897103, 0.744663664144243, 0.7271458610317483, 0.7352780944139626, 0.7832437733699391]
    
BOW de 1500:
Error de entrenamiento: [0.9183495169859025, 0.9240663680071464, 0.9222410689086324, 0.9219469596125256, 0.9192158710855989]
Error de test: [0.7578230517950811, 0.7448159847015519, 0.7385114704860527, 0.7146224519861784, 0.7792601353429577]
    
BOW de 2000
Error de entrenamiento: [0.9209437005267092, 0.9282109885651806, 0.9250446884068524, 0.9255420048144667, 0.9223689527692223]
Error de test: [0.7634241416737515, 0.7429827505329677, 0.7248590220271185, 0.7173043482076129, 0.7790212053446008]
    
BOw de 2500(el mejor por ahora):
Error de entrenamiento: [0.9241660802251935, 0.9297554251416461, 0.9260564112760288, 0.9282990612675541, 0.924205956762895]
Error de test: [0.7668899969828157, 0.7487910499559515, 0.7260748150134895, 0.7417172048245112, 0.7812515158429165]
    
BOW de 5000
Error de entrenamiento: [0.9288016949428641, 0.9352246030036342, 0.9307978723945535, 0.9317218887646183, 0.9294494209070807]
Error de test: [0.7642922911156823, 0.7451959847036329, 0.7239359572326771, 0.7232840880433062, 0.7827675153973228]
    
Me quedo con el de 2500, despues lo tunneo

# Entreno y calculo el feature de prediccion con kfolds

In [ ]:
test_error=[]
train_pred = []
print('iter')
for train_index,test_index in kf.split(train_limpio_normalizado):
    x_train,x_test = train_limpio_normalizado.loc[train_index], train_limpio_normalizado.loc[test_index]
    y_train,y_test = train_y[train_index], train_y[test_index]

    model = svm.SVC(C=80,gamma=0.001)
    model.fit(x_train, y_train)

    #error de test:
    pred = model.predict(x_test)
    
    train_pred.extend(y_pred_binary)
    
    error = f1_score(y_test.values, y_pred_binary, average='macro')
    print(error)
    test_error.append(error)
    
test_error

In [ ]:
# Lo guardo
train = pd.read_csv("train.csv",encoding = "ISO-8859-1")
train.target = train_pred
train[['id','target']].to_csv('pred_train_svm.csv', index=False)

# Realizo la prediccion

In [118]:
clf = svm.SVC()
clf.fit(train_limpio_normalizado, train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [119]:
y_pred = clf.predict(test_limpio_normalizado)

In [120]:
sample.target = y_pred

In [121]:
sample.to_csv('prediccion_SVM_2500.csv', index=False)

In [122]:
sample.target.value_counts()

0    2192
1    1071
Name: target, dtype: int64